# Week 2 Exercise -- Country Explorer

A multi-modal chatbot with real country data, SVG map outlines, and text-to-speech.

**Skills:** Gradio Blocks UI, streaming, tool/function calling, multi-model switching, TTS audio

In [ ]:
import os
import json
import tempfile
import requests
from dotenv import load_dotenv
from openai import OpenAI
from IPython.display import display, HTML
import gradio as gr

In [ ]:
# initialization

load_dotenv(override=True)

openai_client = OpenAI()
openrouter_client = OpenAI(
    base_url="https://openrouter.ai/api/v1",
    api_key=os.getenv("OPENROUTER_API_KEY")
)

MODEL = "gpt-4.1-mini"

In [ ]:
# model switching configuration

model_config = {
    "GPT": (openai_client, "gpt-4.1-mini"),
    "Claude": (openrouter_client, "anthropic/claude-3.5-haiku"),
    "Gemini": (openrouter_client, "google/gemini-2.5-flash-lite"),
}

---
## Tools

Two tools the LLM can call: one for real country data, one for fetching SVG map outlines.

In [ ]:
# Tool 1: fetch verified country data from REST Countries API

API_URL = "https://restcountries.com/v3.1/name"
API_FIELDS = "name,capital,population,languages,currencies,region,subregion,flag,timezones,cca2"

def get_country_data(country_name):
    try:
        resp = requests.get(
            f"{API_URL}/{country_name}",
            params={"fields": API_FIELDS},
            timeout=10
        )
        if resp.status_code != 200:
            return f"Could not find a country called '{country_name}'."
        data = resp.json()[0]
        languages = ", ".join(data.get("languages", {}).values())
        currencies = ", ".join(
            f"{v['name']} ({v['symbol']})" for v in data.get("currencies", {}).values()
        )
        return (
            f"{data.get('flag', '')} {data['name']['official']}\n"
            f"Capital: {data.get('capital', ['Unknown'])[0]}\n"
            f"Population: {data['population']:,}\n"
            f"Languages: {languages}\n"
            f"Currencies: {currencies}\n"
            f"Region: {data.get('region', '')} -- {data.get('subregion', '')}\n"
            f"Timezones: {', '.join(data.get('timezones', []))}"
        )
    except requests.RequestException as e:
        return f"Network error: {e}"

In [ ]:
get_country_data("Rwanda")

In [ ]:
get_country_data("Wakanda")

In [ ]:
# Tool 2: fetch real SVG map outline from mapsicon (djaiss/mapsicon on GitHub)

MAP_SVG_URL = "https://raw.githubusercontent.com/djaiss/mapsicon/master/all/{cca2}/vector.svg"
CCA2_API = "https://restcountries.com/v3.1/name/{name}?fields=cca2"

def generate_country_map(country_name):
    try:
        resp = requests.get(CCA2_API.format(name=country_name), timeout=10)
        if resp.status_code != 200:
            return None
        cca2 = resp.json()[0]["cca2"].lower()
        svg_resp = requests.get(MAP_SVG_URL.format(cca2=cca2), timeout=10)
        if svg_resp.status_code != 200:
            return None
        return svg_resp.text
    except (requests.RequestException, KeyError, IndexError):
        return None

In [ ]:
svg = generate_country_map("Burundi")
if svg:
    display(HTML(svg))
else:
    print("Could not fetch map")

---
## Tool Schemas

The JSON definitions that tell the LLM what tools are available.

In [ ]:
country_data_function = {
    "name": "get_country_data",
    "description": "Get verified country info: capital, population, languages, currencies, region",
    "parameters": {
        "type": "object",
        "properties": {
            "country_name": {
                "type": "string",
                "description": "The country name"
            }
        },
        "required": ["country_name"],
        "additionalProperties": False
    }
}

In [ ]:
country_map_function = {
    "name": "generate_country_map",
    "description": "Fetch an SVG map showing a country's geographic outline",
    "parameters": {
        "type": "object",
        "properties": {
            "country_name": {
                "type": "string",
                "description": "The country name"
            }
        },
        "required": ["country_name"],
        "additionalProperties": False
    }
}

In [ ]:
tools = [
    {"type": "function", "function": country_data_function},
    {"type": "function", "function": country_map_function},
]

---
## Chat Engine

The system prompt, tool handler, and agentic chat loop.

In [ ]:
system_message = """You are a knowledgeable and friendly country expert.
When asked about a country, use get_country_data for verified facts
and generate_country_map for a visual map outline.
The map is displayed in a separate panel -- do not mention or reference map generation in your response.
Present the data engagingly and include a fun fact the user might not know.
If the question isn't about a country, respond normally but mention your specialty."""

In [ ]:
def handle_tool_calls(message):
    responses = []
    svg = None
    for tc in message.tool_calls:
        args = json.loads(tc.function.arguments)
        name = tc.function.name
        if name == "get_country_data":
            result = get_country_data(args["country_name"])
        elif name == "generate_country_map":
            svg = generate_country_map(args["country_name"])
            result = "Map displayed to the user." if svg else "Map unavailable."
        else:
            result = "Unknown tool"
        responses.append({"role": "tool", "content": result, "tool_call_id": tc.id})
    return responses, svg

In [ ]:
def chat(history, model_name):
    client, model = model_config[model_name]
    messages = [{"role": "system", "content": system_message}]
    messages += [{"role": h["role"], "content": h["content"]} for h in history]

    svg_result = None

    # agentic tool-calling loop
    response = client.chat.completions.create(model=model, messages=messages, tools=tools)

    while response.choices[0].finish_reason == "tool_calls":
        tool_msg = response.choices[0].message
        tool_responses, svg = handle_tool_calls(tool_msg)
        if svg:
            svg_result = svg
        messages.append(tool_msg)
        messages.extend(tool_responses)
        response = client.chat.completions.create(model=model, messages=messages, tools=tools)

    reply = response.choices[0].message.content or ""
    return reply, svg_result

---
## Multi-modal: Text-to-Speech

In [ ]:
def talker(text):
    try:
        response = openai_client.audio.speech.create(
            model="gpt-4o-mini-tts",
            voice="onyx",
            input=text[:500]
        )
        with tempfile.NamedTemporaryFile(suffix=".mp3", delete=False) as f:
            f.write(response.content)
            return f.name
    except Exception:
        return None

---
## Gradio UI

`gr.Blocks` for custom layout: chat panel + SVG map + audio + model switcher.

In [ ]:
# callbacks

def put_message_in_chatbot(message, history):
    if not message.strip():
        return message, history
    return "", history + [{"role": "user", "content": message}]


def respond(history, model_name):
    if not history or history[-1]["role"] != "user":
        yield history, None, None
        return

    reply, svg_result = chat(history, model_name)

    svg_html = (
        f'<div style="display:flex;justify-content:center;align-items:center;'
        f'height:100%;max-width:350px;max-height:350px;margin:auto">{svg_result}</div>'
        if svg_result else None
    )

    # stream the reply to the chatbot
    streamed = ""
    for char in reply:
        streamed += char
        yield history + [{"role": "assistant", "content": streamed}], svg_html, None

    # generate audio after text finishes
    audio = talker(reply)
    yield history + [{"role": "assistant", "content": reply}], svg_html, audio

In [ ]:
# UI definition

with gr.Blocks(title="Country Explorer") as ui:
    gr.Markdown("### Country Explorer")

    with gr.Row():
        chatbot = gr.Chatbot(height=400, type="messages")
        map_display = gr.HTML(
            value='<div style="height:350px;display:flex;align-items:center;'
                  'justify-content:center;color:#888">Map appears here</div>'
        )

    with gr.Row():
        audio_output = gr.Audio(autoplay=True)

    with gr.Row():
        message = gr.Textbox(label="Ask about a country:", placeholder="Tell me about Japan...", scale=3)
        model_dropdown = gr.Dropdown(list(model_config.keys()), value="GPT", label="Model", scale=1)

    message.submit(
        put_message_in_chatbot, [message, chatbot], [message, chatbot]
    ).then(
        respond, [chatbot, model_dropdown], [chatbot, map_display, audio_output]
    )

ui.launch(inbrowser=True)